### Installing Required Libraries

In this cell, we install the necessary Python libraries for our project. Specifically, we ensure that `pandas` is installed for data manipulation.

In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


### Installing maplotlib and seaborn because they are not installed on Python ambient

In [2]:
import sys
print(sys.executable)


C:\Users\capor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe


In [3]:
import sys
!{sys.executable} -m pip install matplotlib seaborn


Defaulting to user installation because normal site-packages is not writeable


### Importing Necessary Libraries

We import all the required libraries for the project, including `pymongo` for MongoDB interactions, `pandas` for data manipulation, `re` and `json` for data processing, and `datetime` for handling date and time data.

In [4]:
import pymongo
import pandas as pd
import re
import json
from datetime import datetime
from pprint import pprint

### Establishing Connection to MongoDB

We establish a connection to the MongoDB server running on `localhost` at the default port `27017`. This connection allows us to interact with the MongoDB instance for data storage and retrieval.

In [5]:
# Connection to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['BankChurners']

# Verify connection
print("Connessione a MongoDB stabilita:", client)
print("Database attuale:", db)

Connessione a MongoDB stabilita: MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database attuale: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BankChurners')


### Loading the Dataset

We load the `BankChurners.csv` dataset into a pandas DataFrame. This dataset contains information about credit card customers, which we will analyze and store in MongoDB.

In [6]:
df = pd.read_csv("BankChurners.csv")

### Displaying the First Few Rows of the Dataset

To get an initial understanding of the dataset, we display the first five rows. This helps in verifying that the data has been loaded correctly and provides a glimpse of the data structure.

In [7]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


### Displaying the Last Few Rows of the Dataset

Similarly, we display the last five rows of the dataset to inspect the tail end of the data. This can help identify any anomalies or issues present towards the end of the dataset.

In [8]:
df.tail()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294
10126,714337233,Attrited Customer,43,F,2,Graduate,Married,Less than $40K,Silver,25,...,10388.0,1961,8427.0,0.703,10294,61,0.649,0.189,0.996620,0.003377


### Checking the Shape of the Dataset

We check the dimensions of the DataFrame to understand the number of records and features present in the dataset. This information is crucial for planning data processing and storage strategies.

In [9]:
df.shape

(10127, 23)

### Cleaning and Preprocessing of data

Before proceeding, it is essential to clean and preprocess the data to ensure quality and consistency. This includes handling missing values, converting data types, and removing duplicates.

In [10]:
# Verifica dei valori mancanti
print("Valori mancanti nel dataset:")
print(df.isnull().sum())

# Rimozione dei duplicati
df.drop_duplicates(inplace=True)

# Gestione dei valori mancanti
# Riempimento dei valori mancanti con la mediana per le colonne numeriche
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Per le colonne categoriali, riempimento con il valore più frequente
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Verifica dopo la pulizia
print("\nValori mancanti dopo la pulizia:")
print(df.isnull().sum())


Valori mancanti nel dataset:
CLIENTNUM                                                                                                                             0
Attrition_Flag                                                                                                                        0
Customer_Age                                                                                                                          0
Gender                                                                                                                                0
Dependent_count                                                                                                                       0
Education_Level                                                                                                                       0
Marital_Status                                                                                                                        0
Income_Category    

### Cleaning and Preprocessing of data 2

I saw from checking the dataset head and tail, that value of income it's really hard to manage for the analisys so we perform some change of the dataset itself.

## 1. Analisi dei Valori Esistenti nella Colonna Income_Category
Prima di procedere con la trasformazione, è importante conoscere tutti i valori unici presenti nella colonna.

In [11]:
# Visualizza i valori unici nella colonna 'Income_Category'
print("Valori unici in 'Income_Category':")
print(df['Income_Category'].unique())


Valori unici in 'Income_Category':
['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']


#### Verify how much record has got Unknown income so that I can choose if delete those record or maintain

In [12]:
# Identificazione dei record con 'Unknown' in 'Income_Category'
unknown_income = df[df['Income_Category'] == 'Unknown']
print(f"Numero di record con 'Unknown' in 'Income_Category': {unknown_income.shape[0]}")

Numero di record con 'Unknown' in 'Income_Category': 1112


##### They are really too much to delete them, so we maintain giving to each income different income_category to not loose othere information for further analysis

Cleaning Steps:

Remove Extra Spaces: Some entries like '$120K +' have an extra space before the +. 
Convert to lower cases

In [13]:
# Remove leading and trailing spaces
df['Income_Category'] = df['Income_Category'].str.strip()

# Convert to lowercase for consistent mapping
df['Income_Category'] = df['Income_Category'].str.lower()

# Replace any multiple spaces with a single space
df['Income_Category'] = df['Income_Category'].str.replace(r'\s+', ' ', regex=True)

# Remove spaces before '+' signs
df['Income_Category'] = df['Income_Category'].str.replace(r' +\+', '+', regex=True)


#### 2. Mapping Definition

Based on the unique values, we can create a mapping that assigns a numerical range to each category. This will make aggregation and analysis easier.

Example Mapping:

| Original        | Income_Min | Income_Max | Income_Category_Code |
|-----------------|------------|------------|----------------------|
| Less than $40K  | 0          | 40,000     | 1                    |
| $40K - $60K     | 40,000     | 60,000     | 2                    |
| $60K - $80K     | 60,000     | 80,000     | 3                    |
| $80K - $120K    | 80,000     | 120,000    | 4                    |
| $120K+          | 120,000    | None       | 5                    |

**Notes:**

- **Income_Min**: Minimum value of the income range.
- **Income_Max**: Maximum value of the income range (None for incomes above $120K).
- **Income_Category_Code**: Numerical code to facilitate aggregations.


## 3. Applicazione della Mappatura al DataFrame
Implementiamo la mappatura utilizzando una funzione personalizzata e creiamo nuove colonne per rappresentare i valori numerici.

In [14]:
# Definition of the updated mapping function
def map_income_category(income):
    """
    Maps the 'Income_Category' to numerical codes and defines income ranges.
    
    Parameters:
    income (str): The income category string.
    
    Returns:
    dict: A dictionary with 'Income_Min', 'Income_Max', and 'Income_Category_Code'.
    """
    if income == 'less than $40k':
        return {'Income_Min': 0, 'Income_Max': 40000, 'Income_Category_Code': 1}
    elif income == '$40k - $60k':
        return {'Income_Min': 40000, 'Income_Max': 60000, 'Income_Category_Code': 2}
    elif income == '$60k - $80k':
        return {'Income_Min': 60000, 'Income_Max': 80000, 'Income_Category_Code': 3}
    elif income == '$80k - $120k':
        return {'Income_Min': 80000, 'Income_Max': 120000, 'Income_Category_Code': 4}
    elif income == '$120k+':
        return {'Income_Min': 120000, 'Income_Max': None, 'Income_Category_Code': 5}
    elif income == 'unknown':
        return {'Income_Min': None, 'Income_Max': None, 'Income_Category_Code': 6}
    else:
        return {'Income_Min': None, 'Income_Max': None, 'Income_Category_Code': None}

# 1) Apply the updated mapping function
income_mapped = df['Income_Category'].apply(map_income_category).apply(pd.Series)

# 2) Concat new columns to the DataFrame
df = pd.concat([df, income_mapped], axis=1)

# 3) Drop the original 'Income_Category' column
df.drop('Income_Category', axis=1, inplace=True)

# 4) Verify the mapping
print(df[['Income_Category_Code', 'Income_Min', 'Income_Max']].head())

# 5) Create the 'Income_Info' sub-document,
#    handling NaN → None and int(...) conversion in the same step
def build_income_subdoc(row):
    cat_code = row["Income_Category_Code"]
    i_min = row["Income_Min"]
    i_max = row["Income_Max"]
    
    # If cat_code is NaN, use None; else int(...)
    if pd.isna(cat_code):
        cat_code = None
    else:
        cat_code = int(cat_code)
    
    # If i_min or i_max are NaN, use None; else int(...)
    if pd.isna(i_min):
        i_min = None
    else:
        i_min = int(i_min)
        
    if pd.isna(i_max):
        i_max = None
    else:
        i_max = int(i_max)
    
    return {
        "Category_Code": cat_code,
        "Min": i_min,
        "Max": i_max
    }

df["Income_Info"] = df.apply(build_income_subdoc, axis=1)

# 6) Drop the separate columns now that they are nested in "Income_Info"
df.drop(["Income_Category_Code", "Income_Min", "Income_Max"], axis=1, inplace=True)

print("\nStruttura delle prime 5 righe con Income_Info annidato:")
print(df.head(5))


   Income_Category_Code  Income_Min  Income_Max
0                   3.0     60000.0     80000.0
1                   1.0         0.0     40000.0
2                   4.0     80000.0    120000.0
3                   1.0         0.0     40000.0
4                   3.0     60000.0     80000.0

Struttura delle prime 5 righe con Income_Info annidato:
   CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0  768805383  Existing Customer            45      M                3   
1  818770008  Existing Customer            49      F                5   
2  713982108  Existing Customer            51      M                3   
3  769911858  Existing Customer            40      F                4   
4  709106358  Existing Customer            40      M                3   

  Education_Level Marital_Status Card_Category  Months_on_book  \
0     High School        Married          Blue              39   
1        Graduate         Single          Blue              44   
2        Graduate 

4. Applicazione della Mappatura Aggiornata
Applichiamo la funzione aggiornata per trasformare la colonna Income_Category.

### Splitting the Dataset into Customers and Transactions

For better data management and to align with our MongoDB collections, we split the original DataFrame into two separate DataFrames: `customers_data` and `transactions_data`. This separation allows for more organized storage and targeted queries in MongoDB.

In [15]:
# 1. Seleziona le colonne per 'customers_data'
customer_columns = [
    "CLIENTNUM", "Attrition_Flag", "Customer_Age", "Gender", "Dependent_count", 
    "Education_Level", "Marital_Status", "Card_Category", "Months_on_book",
    "Total_Relationship_Count", "Months_Inactive_12_mon", "Contacts_Count_12_mon",
    "Income_Info",  # <-- Sostituisce le colonne Income_Min, Income_Max, Income_Category_Code
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", 
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
]

customers_data = df[customer_columns].copy()

# 2. Seleziona le colonne per 'transactions_data'
transaction_columns = [
    "CLIENTNUM", "Total_Trans_Amt", "Total_Revolving_Bal", "Credit_Limit", 
    "Total_Amt_Chng_Q4_Q1", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio"
]

transactions_data = df[transaction_columns].copy()

# 3. Verifica le dimensioni dei DataFrame creati
print("Dimensioni di customers_data:", customers_data.shape)
print("Dimensioni di transactions_data:", transactions_data.shape)


Dimensioni di customers_data: (10127, 15)
Dimensioni di transactions_data: (10127, 7)


### Setting Up MongoDB Collections with Schema Validation

We set up two MongoDB collections: `customers` and `transactions`. Before creating these collections, we check if they already exist and drop them to ensure a clean slate. We also define schema validation rules using `$jsonSchema` to enforce data integrity, ensuring that the inserted documents adhere to the specified structure and data types.

In [16]:
import pymongo

# Connessione a MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['BankChurners']

# Cancellazione delle collezioni esistenti se presenti
if "customers" in db.list_collection_names():
    db.customers.drop()
    print("Collection 'customers' dropped.")
if "transactions" in db.list_collection_names():
    db.transactions.drop()
    print("Collection 'transactions' dropped.")

# Creazione della collezione 'customers' con schema validator
db.create_collection("customers", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": [
            "CLIENTNUM", 
            "Attrition_Flag", 
            "Customer_Age", 
            "Gender", 
            "Dependent_count", 
            "Education_Level", 
            "Marital_Status", 
            "Card_Category",
            "Income_Info"    # Ora richiediamo 'Income_Info' invece di Income_Min/Max e Category_Code
        ],
        "properties": {
            "CLIENTNUM": {
                "bsonType": "int",
                "description": "must be an integer and is required"
            },
            "Attrition_Flag": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "Customer_Age": {
                "bsonType": "int",
                "description": "must be an integer and is required"
            },
            "Gender": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "Dependent_count": {
                "bsonType": "int",
                "description": "must be an integer and is required"
            },
            "Education_Level": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "Marital_Status": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "Card_Category": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "Income_Info": {
                "bsonType": "object",
                "description": "Nested document for income ranges",
                "required": ["Category_Code", "Min", "Max"],
                "properties": {
                    "Category_Code": { "bsonType": "int", "description": "must be an integer" },
                    "Min": { "bsonType": ["int", "null"], "description": "must be an integer or null" },
                    "Max": { "bsonType": ["int", "null"], "description": "must be an integer or null" }
                }
            },

            # Altri campi opzionali (non in required) che puoi includere:
            "Months_on_book": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Total_Relationship_Count": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Months_Inactive_12_mon": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Contacts_Count_12_mon": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1": {
                "bsonType": "double",
                "description": "must be a double"
            },
            "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2": {
                "bsonType": "double",
                "description": "must be a double"
            }
        }
    }
})
print("Collection 'customers' created successfully.")

# Creazione della collezione 'transactions' con schema validator
db.create_collection("transactions", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": [
            "CLIENTNUM", 
            "Total_Trans_Amt", 
            "Total_Revolving_Bal", 
            "Credit_Limit", 
            "Total_Amt_Chng_Q4_Q1", 
            "Total_Ct_Chng_Q4_Q1", 
            "Avg_Utilization_Ratio"
        ],
        "properties": {
            "CLIENTNUM": {
                "bsonType": "int",
                "description": "must be an integer and is required"
            },
            "Total_Trans_Amt": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            "Total_Revolving_Bal": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            "Credit_Limit": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            "Total_Amt_Chng_Q4_Q1": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            "Total_Ct_Chng_Q4_Q1": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            "Avg_Utilization_Ratio": {
                "bsonType": "double",
                "description": "must be a double and is required"
            },
            # Campi opzionali
            "Months_on_book": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Total_Relationship_Count": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Months_Inactive_12_mon": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Contacts_Count_12_mon": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Dependent_count": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1": {
                "bsonType": "double",
                "description": "must be a double"
            },
            "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2": {
                "bsonType": "double",
                "description": "must be a double"
            }
        }
    }
})
print("Collection 'transactions' created successfully.")


Collection 'customers' dropped.
Collection 'transactions' dropped.
Collection 'customers' created successfully.
Collection 'transactions' created successfully.


### Ensuring Correct Data Format and Inserting into MongoDB

Before inserting the data into MongoDB, we ensure that the data types of the DataFrame columns match the schema defined in MongoDB. Specifically, we convert the necessary columns to `float` to comply with the `double` type requirements. After formatting, we convert the DataFrames to dictionaries and attempt to insert them into the respective MongoDB collections. Proper exception handling is implemented to catch and report any insertion errors.

In [17]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Card_Category,Months_on_book,Total_Relationship_Count,...,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,Income_Info
0,768805383,Existing Customer,45,M,3,High School,Married,Blue,39,5,...,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991,"{'Category_Code': 3, 'Min': 60000, 'Max': 80000}"
1,818770008,Existing Customer,49,F,5,Graduate,Single,Blue,44,6,...,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994,"{'Category_Code': 1, 'Min': 0, 'Max': 40000}"
2,713982108,Existing Customer,51,M,3,Graduate,Married,Blue,36,4,...,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998,"{'Category_Code': 4, 'Min': 80000, 'Max': 120000}"
3,769911858,Existing Customer,40,F,4,High School,Unknown,Blue,34,3,...,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987,"{'Category_Code': 1, 'Min': 0, 'Max': 40000}"
4,709106358,Existing Customer,40,M,3,Uneducated,Married,Blue,21,5,...,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998,"{'Category_Code': 3, 'Min': 60000, 'Max': 80000}"


In [18]:
# verify which columns are in the two collections
print("Colonne di customers_data:", customers_data.columns.tolist())
print("Colonne di transactions_data:", transactions_data.columns.tolist())


Colonne di customers_data: ['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count', 'Education_Level', 'Marital_Status', 'Card_Category', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Income_Info', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']
Colonne di transactions_data: ['CLIENTNUM', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Credit_Limit', 'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']


In [19]:
# 1) Convert columns to int, float, or str only if they exist in each DataFrame

# --- INT CONVERSION ---
int_columns_customers = [
    "CLIENTNUM", "Customer_Age", "Dependent_count",
    "Months_on_book", "Total_Relationship_Count",
    "Months_Inactive_12_mon", "Contacts_Count_12_mon"
]
existing_int_cols_customers = [col for col in int_columns_customers if col in customers_data.columns]
customers_data[existing_int_cols_customers] = customers_data[existing_int_cols_customers].astype(int)

int_columns_transactions = [
    "CLIENTNUM"
    # Add more if they exist in transactions_data
]
existing_int_cols_transactions = [col for col in int_columns_transactions if col in transactions_data.columns]
transactions_data[existing_int_cols_transactions] = transactions_data[existing_int_cols_transactions].astype(int)

# --- FLOAT CONVERSION ---
# These fields must be double per your schema
float_columns_customers = [
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
]
existing_float_cols_customers = [col for col in float_columns_customers if col in customers_data.columns]
customers_data[existing_float_cols_customers] = customers_data[existing_float_cols_customers].astype(float)

float_columns_transactions = [
    "Total_Trans_Amt", "Total_Revolving_Bal", "Credit_Limit",
    "Total_Amt_Chng_Q4_Q1", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio"
]
existing_float_cols_transactions = [col for col in float_columns_transactions if col in transactions_data.columns]
transactions_data[existing_float_cols_transactions] = transactions_data[existing_float_cols_transactions].astype(float)

# --- STRING CONVERSION ---
string_columns_customers = [
    "Attrition_Flag", "Gender", "Education_Level", "Marital_Status",
    "Card_Category"
]
existing_string_cols_customers = [col for col in string_columns_customers if col in customers_data.columns]
customers_data[existing_string_cols_customers] = customers_data[existing_string_cols_customers].astype(str)

string_columns_transactions = [
    # If you have any string columns in transactions_data, list them here
]
existing_string_cols_transactions = [col for col in string_columns_transactions if col in transactions_data.columns]
transactions_data[existing_string_cols_transactions] = transactions_data[existing_string_cols_transactions].astype(str)

# 2) Insert data into MongoDB with exception handling
try:
    db.customers.delete_many({})
    db.transactions.delete_many({})
    
    customers_dict = customers_data.to_dict("records")
    transactions_dict = transactions_data.to_dict("records")
    
    db.customers.insert_many(customers_dict)
    db.transactions.insert_many(transactions_dict)
    
    print("Data inserted successfully into MongoDB.")
except pymongo.errors.BulkWriteError as bwe:
    print("Bulk write error occurred during data insertion:")
    for error in bwe.details['writeErrors']:
        print(f"Error at index {error['index']}: {error['errmsg']}")
except Exception as e:
    print(f"An unexpected error occurred during insertion: {e}")


Data inserted successfully into MongoDB.


### Verifying Data Insertion by Fetching Samples from MongoDB

After inserting the data, we fetch and print the first five documents from both the `customers` and `transactions` collections in MongoDB. This verification step ensures that the data has been inserted correctly and adheres to the defined schema.

In [ ]:
# Fetch and print data to verify
print(list(db.customers.find().limit(5)))
print(list(db.transactions.find().limit(5)))

### Summary and Next Steps

In this notebook, we successfully loaded the Bank Churners dataset, processed it to fit our MongoDB schema, and inserted it into MongoDB collections with schema validation. The data insertion was verified by fetching sample documents from the collections.

**Next Steps:**
1. **Customer Segmentation:** Implement aggregation pipelines in MongoDB to segment customers based on age, gender, education level, and income category.
2. **Customer Status Analysis:** Analyze the factors contributing to account closures by examining the `Attrition_Flag` and transaction patterns.
3. **Financial Health Assessment:** Assess the financial health of customers by analyzing credit limits, revolving balances, and credit utilization ratios.
4. **Visualization:** Utilize libraries like `matplotlib` and `seaborn` to visualize the insights derived from the data.
5. **Advanced Analysis:** Apply machine learning techniques to predict customer attrition and identify high-risk customers.

### Additional Enhancements

To further enhance your project, consider implementing the following:

1. **Environment Management**

   Create a `requirements.txt` file to manage project dependencies. This facilitates easy setup of the project environment.

   ```bash
   # requirements.txt
   pandas
   pymongo
   ```

   You can install the dependencies using:

   ```bash
   pip install -r requirements.txt
   ```

2. **Data Validation and Cleaning**

   Before inserting data into MongoDB, perform comprehensive data validation and cleaning to ensure data quality.

   ```python
   # Check for missing values
   print("Missing values in customers_data:")
   print(customers_data.isnull().sum())
   
   print("
Missing values in transactions_data:")
   print(transactions_data.isnull().sum())
   
   # Handle missing values
   customers_data.dropna(inplace=True)  # Example: Drop rows with missing values
   transactions_data.fillna(transactions_data.median(), inplace=True)  # Example: Fill missing with median
   ```

3. **Indexing in MongoDB**

   Create indexes on commonly queried fields to improve query performance.

   ```python
   # Create index on CLIENTNUM for faster queries
   db.customers.create_index([("CLIENTNUM", pymongo.ASCENDING)], unique=True)
   db.transactions.create_index([("CLIENTNUM", pymongo.ASCENDING)])
   ```

4. **Aggregation Pipelines for Analysis**

   Implement MongoDB aggregation pipelines to perform the analyses outlined in your project proposal.

#### a. Customer Segmentation

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": {
                "Age": "$Customer_Age",
                "Gender": "$Gender",
                "Education_Level": "$Education_Level",
                "Income_Category": "$Income_Category"
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]

segmentation = list(db.customers.aggregate(pipeline))
pprint(segmentation)

# Convert to DataFrame for visualization
segmentation_df = pd.DataFrame(segmentation)
segmentation_df = segmentation_df['_id'].apply(pd.Series).join(segmentation_df['count'])

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(data=segmentation_df, x='Gender', hue='Income_Category')
plt.title('Customer Segmentation by Gender and Income Category')
plt.xlabel('Gender')
plt.ylabel('Number of Customers')
plt.show()


#### b. Customer Status Analysis


In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "transactions",
            "localField": "CLIENTNUM",
            "foreignField": "CLIENTNUM",
            "as": "transactions_info"
        }
    },
    {
        "$match": {
            "Attrition_Flag": "Attrited Customer"
        }
    },
    {
        "$unwind": "$transactions_info"
    },
    {
        "$project": {
            "CLIENTNUM": 1,
            "Total_Trans_Amt": "$transactions_info.Total_Trans_Amt",
            "Total_Revolving_Bal": "$transactions_info.Total_Revolving_Bal",
            "Credit_Limit": "$transactions_info.Credit_Limit",
            "Avg_Utilization_Ratio": "$transactions_info.Avg_Utilization_Ratio"
        }
    }
]

attrition_analysis = list(db.customers.aggregate(pipeline))
pprint(attrition_analysis)

# Convert to DataFrame for analysis
attrition_df = pd.DataFrame(attrition_analysis)

# Visualization
sns.histplot(attrition_df['Avg_Utilization_Ratio'], kde=True)
plt.title('Distribution of Credit Utilization Ratio Among Attrited Customers')
plt.xlabel('Average Utilization Ratio')
plt.ylabel('Frequency')
plt.show()

# Correlation Heatmap
correlation = attrition_df[['Total_Trans_Amt', 'Total_Revolving_Bal', 'Credit_Limit', 'Avg_Utilization_Ratio']].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Between Financial Variables')
plt.show()

#### c. Financial Health Assessment

In [ ]:
pipeline = [
    {
        "$project": {
            "CLIENTNUM": 1,
            "Credit_Limit": 1,
            "Total_Revolving_Bal": 1,
            "Avg_Utilization_Ratio": 1
        }
    }
]

financial_health = list(db.transactions.aggregate(pipeline))
financial_health_df = pd.DataFrame(financial_health)

# Visualization
sns.scatterplot(data=financial_health_df, x='Credit_Limit', y='Total_Revolving_Bal', hue='Avg_Utilization_Ratio')
plt.title('Credit Limit vs. Total Revolving Balance')
plt.xlabel('Credit Limit')
plt.ylabel('Total Revolving Balance')
plt.show()

# Identifying High-Risk Customers
high_risk_customers = financial_health_df[
    (financial_health_df['Avg_Utilization_Ratio'] > 0.75) | 
    (financial_health_df['Total_Revolving_Bal'] > financial_health_df['Credit_Limit'] * 0.8)
]

print("High-Risk Customers:")
print(high_risk_customers)


### Conclusion

In this notebook, we successfully:

1. **Installed and Imported Libraries:** Ensured all necessary libraries are available for data manipulation and MongoDB interactions.
2. **Connected to MongoDB:** Established a connection to the MongoDB server and set up collections with schema validation to maintain data integrity.
3. **Loaded and Inspected Data:** Loaded the dataset, displayed its structure, and verified the number of records and features.
4. **Processed and Inserted Data:** Split the dataset into relevant sections, ensured correct data types, and inserted the data into MongoDB with proper error handling.
5. **Verified Data Insertion:** Fetched and displayed sample documents from MongoDB to confirm successful data insertion.
6. **Outlined Next Steps:** Provided a roadmap for further analysis, including customer segmentation, status analysis, and financial health assessment.

By following the structured approach and implementing best practices, the project is well-positioned to deliver meaningful insights into customer behaviors and factors influencing account closures.